Iterative version

```
FUNCTION Throw2Score(throw: STRING) RETURNS INTEGER
    IF throw = "X" THEN
        RETURN 10
    ELSE:
        RETURN INTEGER(throw) // Convert to integer
    ENDIF
ENDFUNCTION

FUNCTION Calc_Score(throws: STRING) RETURNS INTEGER
// Assume that the input string is valid
    total_score <- 0
    FOR frame = 1 TO 10 DO
        IF throws[1] = "X" THEN // first character of STRING starts at index 1
            frame_score <- SUM(10,Throw2Score(),Throw2Score())
            throws <- SUB_STR(throws, 2, LEN(throws)) 
            // creates a sub string  by removing the first character
        ELSE
            frame_score <- SUM(Throw2Score(throws[1]), Throw2Score(throws[2]) )
            IF frame_score = 10 THEN
                frame_score <- frame_score + Throw2Score(throws[3])
            ENDIF
            throws <- SUB_STR(throws, 3, LEN(throws))   
            // creates a sub string  by removing the first two characters   
        ENDIF
        total_score <- total_score + frame_score
    ENDFOR
    RETURN total_score
ENDFUNCTION
```

In [1]:
## 2.1
def calc_score(throws):

    def throw2score(throw:str)->int:
        if throw == "X":
            return 10
        else:
            return int(throw)
    total_score = 0
    for _ in range(10):
        if throws[0] == "X":
            frame_score = 10 + throw2score(throws[1]) + throw2score(throws[2])
            # 1m
            throws=throws[1:] #1m
        else:
            frame_score = throw2score(throws[0]) + throw2score(throws[1])
            #1m
            if frame_score == 10:
                frame_score += throw2score(throws[2]) #1m
            throws=throws[2:] #1m
        total_score += frame_score
    return total_score

In [2]:
print(
calc_score("X737291XXX2364733"),
calc_score("0580X05X6405819150"),
calc_score("XXXX91X002282XXX")
)

168 97 193


In [3]:
#2.2
def calc_score2(throws, player_name):
    def throw2score(throw:str)->int:
        if throw == "X":
            return 10
        else:
            return int(throw)
    try:
        f = open(f"SCORE_{player_name}.txt", "w")
        ##[1] for initialising variables to store pins hits, frame score and cum. score
        frames = []
        frame_scores = []
        cumulative_scores= [] 
        total_score = 0
        
        ## frame output
        ## frame 1 to 9 outputs for strike, non-strike
        ## frame 10 output for strike/spare, non-strike
        ## [4]
        for f_no in range(10):
            if throws[0] == "X":
                frame_score = 10 + throw2score(throws[1]) + throw2score(throws[2])
                if f_no == 9: 
                    ## last frame for strike
                    frames.append(f"[{throws[0]}|{throws[1]}|{throws[2]}]")
                else:
                    frames.append(f"[{throws[0]:^3}]") 
                    ##frame 1 to 9 for strike
                throws=throws[1:] 
            else:
                frame_score = throw2score(throws[0]) + throw2score(throws[1])
                if frame_score == 10:
                    frame_score += throw2score(throws[2])
                    if f_no == 9:
                        frames.append(f"[{throws[0]}|{throws[1]}|{throws[2]}]")
                        ##spare for last frame 
                    else:
                        frames.append(f"[{throws[0]}|{throws[1]}]")
                        ##spare for frame 1-9
                else:
                    frames.append(f"[{throws[0]}|{throws[1]}]") 
                    ## not spare not strike
                
                throws=throws[2:]
        ## [1] frame_score, cumulative_score output        
            frame_scores.append(f"{frame_score:^5}")
            total_score += frame_score
            cumulative_scores.append(f"{total_score:^5}")
        
        ## [4] file output
        f.write(f"{player_name} Scoresheet\n")
        f.write(f"{'Frame':<12}"+"".join([ f"{f:^5}"for f in  range(1,11)])+"\n" )
        f.write(f"{'Pins hit':<12}"+"".join(frames)+"\n")
        f.write(f"{'FrameScore':<12}" + "".join(frame_scores) +"\n")
        f.write(f"{'Cum. Score':<12}"+"".join(cumulative_scores)+"\n") ##1m
    finally:
        f.close()
    return total_score

In [4]:
calc_score2("X737291XXX2364733","Player 1")
calc_score2("0580X05X6405819150","Player 2")
calc_score2("XXXX91X002282XXX","Player 3")

193

In [6]:
def validate(throws, frame=1):
    if not throws: ## [1] Base case 1: not enough values to compute 10 frames
        print(f"Frame {frame} is invalid")
        return False
    if frame == 10: ## [1] Base case 2: check frame 10
        if throws[0] == "X": ## [1] A Strike needs 2 more throws
            if len(throws) == 3:
                return True
            else:
                return False
        elif len(throws) == 3 and sum( [int(throws[0]), int(throws[1])]) == 10:
            return True ## [1] A spare needs 1 more throw
        elif len(throws) == 2 and sum( [int(throws[0]), int(throws[1])]) < 10:
            return True ## [1] if not strike or spare frame should have only 2 throws
        else:
            print("Frame 10 is invalid")
            return False ## [1]
    else: ## Recusrive case 
        if throws[0] == "X":
            return validate(throws[1:], frame+1) #[1]
        else:
            if throws[:2].isdigit() and ( int(throws[0]) + int(throws[1]) <= 10):
                return validate(throws[2:], frame+1) #[2]
            else:
                print(f"Frame {frame} is invalid")
                return False


In [7]:
print(
validate("XXXX91X00228291X"),
validate("XXXX91X00228290"),
validate("XXXX91X00228291"),
validate("XXXX91")  
)




Frame 10 is invalid
Frame 6 is invalid
True True False False


In [8]:
print(
    	validate("XX90XXX90XXX8"),
        validate("XX90XXX90XXX") ,
        validate("XX90XXX90XXX80"),
        validate("Xz90XXX90XXX80") 
)

Frame 2 is invalid
False False True False


In [ ]:
print(validate("X737291XXX2364733"))
print(validate("X737291XXX2364737"))
print(validate(9))

In [9]:
print(validate("XXXX91X00228291X"))
calc_score2("XXXX91X00228291X","Anon")

True


182